In [24]:
!pip install replicate

In [28]:
!export REPLICATE_API_TOKEN=YOUR_API_TOKEN

In [26]:
import replicate
import pandas as pd

In [29]:
from replicate.client import Client

replicate = Client(api_token="YOUR_API_TOKEN")

In [21]:
train_path = 'path/to/train.csv'
questions = pd.read_csv(train_path)

In [30]:
outputs = []

In [37]:
for index, q in questions.iterrows():
    prompt = f'''<<SYS>>\n Answer the following question with A, B, C, D, or E. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n</s> 
    [INST] Question: {q['prompt']} \n 
    A: {q['A']}\n
    B: {q['B']}\n
    C: {q['C']}\n
    D: {q['D']}\n
    E: {q['E']}\n [/INST] '''
    output = replicate.run(
    "meta/llama-2-7b-chat:13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0",
    input={
        "prompt":prompt,
        'seed':42,
        'max_tokens': 1024,
    }
    )
    outputs.append("".join([i for i in output]))

In [44]:
import re

def extract_answer(text):
    text = text.strip()
    if len(text) == 1:
        return text
    match = re.search(r'\b([ABCDE]):|\(([ABCDE])\)|\b([ABCDE]).|\b([ABCDE])\)|\b([ABCDE]:)', text)
    if match:
        # Returns the first non-None group
        return next(group for group in match.groups() if group is not None)
    else:
        return "No Answer Found"

In [45]:
answers = []
for output in outputs:
    answer = extract_answer(output)
    answers.append(answer)

In [67]:
#save answers to csv
answers_df = pd.DataFrame(answers)
answers_df.to_csv('llama-2-7b-answers.csv')

In [68]:
answers_df.loc[answers_df[0] == 'No Answer Found'].count()

0    29
dtype: int64

In [72]:
answers_df[0].value_counts()

A                  94
No Answer Found    29
C                  25
B                  22
E                  16
D                  14
Name: 0, dtype: int64

In [73]:
questions['answer'].value_counts()

B    48
C    44
D    38
A    37
E    33
Name: answer, dtype: int64

In [74]:
(answers_df[0] == questions['answer']).sum()

47